In [158]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
processed_data = np.load('Full_array.npy')

from tensorflow import keras

import pandas as pd

#The size of the dataset is 10,000; but I only filled the first 9166 with values, the rest are zeros. (shouldn't be counted)
number = 9166
processed_data = processed_data[0:number]

np.random.seed(1)
tf.set_random_seed(1)

In [159]:
#we need to push accuracy to over 90%

#idea number 1: run a set of high percentage accuracy classifications through the NN (nope)

#idea number 2: clean the training set by running through and deleting the bad examples from my data aquisition, were going to 
#set all arrays of zero to be class A. If I feed this NN nothing, I want it to tell me it is odd. (yup)

#first lets see what doing number 2 will get us.

#augment data set manually (worked)

#change LR (Adam is adaptive: so no)

In [160]:
#we want to normalize our iamge data to go from 0 to 1
#processed_data = processed_data/255.
#I actually am not sure how it is scaled. however, some of the pixels are negative for some reason???

#also, this did not help.

In [161]:
#hyperparameters:
HP_allowed_confidence = 0.98
test_train_split = 0.85
batch_size = 128
epoch_number = 20
learning_rate = 0.001
dropout_rate = 0.25
CNL1_filters = 32
CNL1_kernal_size = 5
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 5
MPL2_pool_size = (2,2)
MPL2_strides = 2

In [162]:
#Load in my dataset of targets, targets are strings labels under the name "Class"
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[2,3,4,8,15,21,27], nrows=number)
#galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", nrows=number)
Class = galaxyzoo["gz2class"].values
RA = galaxyzoo['ra'].values
DEC = galaxyzoo['dec'].values
Spiral = galaxyzoo['t01_smooth_or_features_a02_features_or_disk_debiased'].values
Elliptical = galaxyzoo['t01_smooth_or_features_a01_smooth_debiased'].values
Anythingelse = galaxyzoo['t01_smooth_or_features_a03_star_or_artifact_debiased'].values

In [163]:
#I want to take the first character of the Class string and interpret as a integer, ala MNIST example code
dictionary = {'A':int(2),'E':int(1),'S':int(0)}

In [164]:
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [165]:
#go through, find the arrays of zero, set that target to 'A' = 2
for i in range(len(target)):
    if processed_data[i].any() == 0:
        target[i] = 2

In [166]:
from PIL import Image
#rotate each image in the target_images by a random amount
rotations_array = np.empty((np.shape(processed_data)[0],28,28))
for i in range(np.shape(processed_data)[0]):
    degree = np.random.uniform(-180,180,1)
    img = Image.fromarray(processed_data[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
rotated_images = np.append(processed_data, rotations_array,0)
#now append the class labels to the train_target array
rotated_target = np.append(target,target)
print(np.shape(rotated_images))

(18332, 28, 28)


In [167]:
rotations_array = np.empty((np.shape(rotated_images)[0],28,28))
for i in range(np.shape(processed_data)[0]):
    degree = np.random.uniform(-180,180,1)
    img = Image.fromarray(processed_data[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
rotated_images = np.append(rotated_images, rotations_array,0)
#now append the class labels to the train_target array
rotated_target = np.append(rotated_target,rotated_target)
print(np.shape(rotated_images))

(36664, 28, 28)


In [168]:
rotations_array = np.empty((np.shape(rotated_images)[0],28,28))
for i in range(np.shape(processed_data)[0]):
    degree = np.random.uniform(-180,180,1)
    img = Image.fromarray(processed_data[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
rotated_images = np.append(rotated_images, rotations_array,0)
#now append the class labels to the train_target array
rotated_target = np.append(rotated_target,rotated_target)
print(np.shape(rotated_images))

(73328, 28, 28)


In [169]:
"""
rotations_array = np.empty((np.shape(rotated_images)[0],28,28))
for i in range(np.shape(processed_data)[0]):
    degree = np.random.uniform(-180,180,1)
    img = Image.fromarray(processed_data[i])
    rot = img.rotate(degree)
    rot = np.asarray(rot)
    rotations_array[i] = rot
#now append rotations_array to train_images
rotated_images = np.append(rotated_images, rotations_array,0)
#now append the class labels to the train_target array
rotated_target = np.append(rotated_target,rotated_target)
print(np.shape(rotated_images))
print(np.shape(rotated_target))
"""

'\nrotations_array = np.empty((np.shape(rotated_images)[0],28,28))\nfor i in range(np.shape(processed_data)[0]):\n    degree = np.random.uniform(-180,180,1)\n    img = Image.fromarray(processed_data[i])\n    rot = img.rotate(degree)\n    rot = np.asarray(rot)\n    rotations_array[i] = rot\n#now append rotations_array to train_images\nrotated_images = np.append(rotated_images, rotations_array,0)\n#now append the class labels to the train_target array\nrotated_target = np.append(rotated_target,rotated_target)\nprint(np.shape(rotated_images))\nprint(np.shape(rotated_target))\n'

In [170]:
train_split_indice = int(np.round(test_train_split*np.size(rotated_target)))


#split my data between training and test sets
train_target = rotated_target[0:train_split_indice]
test_target = rotated_target[train_split_indice::]
train_images = rotated_images[0:train_split_indice]
test_images = rotated_images[train_split_indice::]

In [171]:
#we want to form a subset of our training examples that are probabilities that they are sorted correctly greater than %90
#we will then train this dataset additional times.
"""
confidence = list()
HP_indice = list()
for i in range(len(train_images)):
    if train_target[i] == 0:
        confidence.append(Spiral[i])
    if train_target[i] == 1:
        confidence.append(Elliptical[i])
    if train_target[i] == 2:
        confidence.append(Anythingelse[i])
    if confidence[i] >= HP_allowed_confidence:
        HP_indice.append(i)
train_weights = np.asarray(confidence)
HP_target = np.empty(len(HP_indice))
HP_images = np.empty((len(HP_indice),28,28))
for i in range(len(HP_indice)):
    HP_target[i] = train_target[HP_indice[i]]
    HP_images[i] = train_images[HP_indice[i]]

#print(len(HP_indice))
HP_images = HP_images.reshape(len(HP_indice),28,28,1)
train_images = train_images.reshape(train_split_indice,28,28,1)
test_images = test_images.reshape(train_split_indice-number,28,28,1)
print(np.shape(train_images))
print(np.shape(HP_images))
"""

#I need this bottom part to come out, but delete it if we wanted the HP set again...
train_images = train_images.reshape(train_split_indice,28,28,1)
test_images = test_images.reshape(np.shape(test_images)[0],28,28,1)

In [172]:
#Next Idea: I want to augment my images to artificially create more data: technically, I should be allowed to do any
#amount of flips and rotations because of the cosmological principle

#if that doesn't work, I will just have to get more data. or add more layers...

#Note: using this generator decreased performance, I dont know how it works. I do know how my manuel offline flips and rotations
#work. And, those offline incresed performance.

"""
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=45,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
"""

"\nfrom keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img\n\ndatagen = ImageDataGenerator(\n        rotation_range=45,\n        #width_shift_range=0.2,\n        #height_shift_range=0.2,\n        #zoom_range=0.2,\n        horizontal_flip=True,\n        vertical_flip=True,\n        fill_mode='nearest')\n"

In [173]:

#define my model, using a CNN with 2 Convolutional layers, 2 max pool layers, 1 dense layer, 1 drop out layer, and another dense layer 
def create_model(dropout_rate=dropout_rate, learning_rate=learning_rate):
    
    model = keras.Sequential([])
    model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters=CNL1_filters,kernel_size=CNL1_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL1_pool_size, strides=MPL1_strides))
    model.add(keras.layers.Conv2D(filters=CNL2_filters,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    model.add(keras.layers.Reshape([7*7*64]))
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    adam = tf.train.AdamOptimizer(learning_rate = learning_rate)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return(model)



In [174]:
#Fit the model with the high probability first, 2-3 epochs increased accuracy just slightly.
#model=create_model(dropout_rate=dropout_rate, learning_rate=learning_rate)
#model.fit(HP_images, HP_target, epochs=2, batch_size=batch_size, verbose=1, shuffle=True)

In [175]:
#model.summary()
#test_loss, test_acc = model.evaluate(test_images.reshape(number - train_split_indice,28,28,1), test_target)
#print('Test accuracy:', test_acc)

In [ ]:
#lets try training one epoch at a time and seeing what epoch when we maximize our accuracy; I fear overtraining

#OTHER than that, my next idea is to change the architecture.... eventually I will need to build a RESnet anyways
model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)

for i in range(epoch_number):
    model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True)
    test_loss, test_acc = model.evaluate(test_images, test_target)
    print('Test accuracy:', test_acc)


Epoch 1/1
43008/62329 [===================>..........] - ETA: 1:09 - loss: 0.7013 - acc: 0.6600

In [ ]:
#model = create_model(dropout_rate=dropout_rate, learning_rate = learning_rate)
#model.fit(train_images, train_target, epochs=1, batch_size=batch_size, verbose=1, shuffle=True

In [ ]:
#model.summary()
test_loss, test_acc = model.evaluate(test_images, test_target)
print('Test accuracy:', test_acc)

#0.848 with the zeros fixed
#0.850 with the HP dataset implemented

#0.822 with augmentation, rescaling
#rescaling seems to worsen my performance; however: why are some of my pixel values negative?

#0.842 with 50 epochs: doesn't seem to be over fitting, perhaps the data augmentation just takes more time to train???

#0.852 with 75 epochs, maybe we need to decrease learning rate past 50???, using generator

#0.857 w/ 75 epochs, using generator

#0.864 with 20 epochs, using offline flips before the data set. dropout = 0.3, no HP set. perhaps we can train for 15 epochs with
#LR  0.001 as usual, then train 5 epochs with LR decreased by factor of 10? 

#I dont think changing my LR will help, it appears Adam already adapts LR on the fly for the situation.
#0.857 w/ droput rate increased to 0.3, will decrease it again... 

#0.864 with 15 epochs, dropout_rate = 0.25, with 124656 images after making two copies of flips and stuff.

#0.852 with 20 epcohs, dropout rate = 0.5, LR = 0.004, batch_size = 128. The accuracy did not seem over trained by the end of the 
#20 epochs, therefore I will run it through another 20 to see what happens